In [1]:
import os
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers
import glob
from tqdm import tqdm
import matplotlib.pyplot as plt


In [2]:
def create_directories():
    os.makedirs('samples', exist_ok=True)
    os.makedirs('models', exist_ok=True)


In [3]:
def load_and_preprocess_images(sharp_path, motion_blur_path, defocus_blur_path, img_size=(256, 256)):
    def load_images(path):
        images = []
        for img_path in glob.glob(os.path.join(path, '*')):
            img = cv2.imread(img_path)
            if img is not None:
                img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
                img = cv2.resize(img, img_size)
                images.append(img)
        if not images:
            raise ValueError(f"No valid images found in {path}")
        return np.array(images)
    
    sharp_images = load_images(sharp_path)
    motion_blur_images = load_images(motion_blur_path)
    defocus_blur_images = load_images(defocus_blur_path)
    
    sharp_images = (sharp_images - 127.5) / 127.5
    motion_blur_images = (motion_blur_images - 127.5) / 127.5
    defocus_blur_images = (defocus_blur_images - 127.5) / 127.5
    
    return sharp_images, motion_blur_images, defocus_blur_images


In [4]:
def make_generator():
    inputs = layers.Input(shape=[256, 256, 3])
    
    x = layers.Conv2D(64, 4, strides=2, padding='same')(inputs)
    x = layers.LeakyReLU()(x)
    
    x = layers.Conv2D(128, 4, strides=2, padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.LeakyReLU()(x)
    
    x = layers.Conv2D(256, 4, strides=2, padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.LeakyReLU()(x)
    
    x = layers.Conv2DTranspose(256, 4, strides=2, padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)
    
    x = layers.Conv2DTranspose(128, 4, strides=2, padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)
    
    x = layers.Conv2DTranspose(64, 4, strides=2, padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)
    
    outputs = layers.Conv2D(3, 4, padding='same', activation='tanh')(x)
    
    return tf.keras.Model(inputs=inputs, outputs=outputs)


In [5]:
def make_discriminator():
    inputs = layers.Input(shape=[256, 256, 3])
    
    x = layers.Conv2D(64, 4, strides=2, padding='same')(inputs)
    x = layers.LeakyReLU()(x)
    
    x = layers.Conv2D(128, 4, strides=2, padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.LeakyReLU()(x)
    
    x = layers.Conv2D(256, 4, strides=2, padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.LeakyReLU()(x)
    
    x = layers.Flatten()(x)
    outputs = layers.Dense(1)(x)
    
    return tf.keras.Model(inputs=inputs, outputs=outputs)


In [6]:
cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=True)

def generator_loss(fake_output):
    return cross_entropy(tf.ones_like(fake_output), fake_output)

def discriminator_loss(real_output, fake_output):
    real_loss = cross_entropy(tf.ones_like(real_output), real_output)
    fake_loss = cross_entropy(tf.zeros_like(fake_output), fake_output)
    return real_loss + fake_loss


In [10]:
class BlurGAN:
    def __init__(self):
        self.generator = make_generator()
        self.discriminator = make_discriminator()
        self.generator_optimizer = tf.keras.optimizers.Adam(2e-4, beta_1=0.5)
        self.discriminator_optimizer = tf.keras.optimizers.Adam(2e-4, beta_1=0.5)
    
    @tf.function
    def train_step(self, sharp_images, blur_images):
        with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
            generated_images = self.generator(blur_images, training=True)
            
            real_output = self.discriminator(sharp_images, training=True)
            fake_output = self.discriminator(generated_images, training=True)
            
            gen_loss = generator_loss(fake_output)
            disc_loss = discriminator_loss(real_output, fake_output)
        
        gen_gradients = gen_tape.gradient(gen_loss, self.generator.trainable_variables)
        disc_gradients = disc_tape.gradient(disc_loss, self.discriminator.trainable_variables)
        
        self.generator_optimizer.apply_gradients(zip(gen_gradients, self.generator.trainable_variables))
        self.discriminator_optimizer.apply_gradients(zip(disc_gradients, self.discriminator.trainable_variables))
        
        return gen_loss, disc_loss
    
    def train(self, sharp_images, blur_images, epochs=50, batch_size=32):
        dataset = tf.data.Dataset.from_tensor_slices((sharp_images, blur_images))\
            .shuffle(1000).batch(batch_size)
        
        for epoch in range(epochs):
            print(f"Epoch {epoch + 1}/{epochs}")
            for batch in tqdm(dataset):
                gen_loss, disc_loss = self.train_step(batch[0], batch[1])
            
            if (epoch + 1) % 5 == 0:
                self.save_samples(epoch + 1, blur_images[:4])
            
            print(f"Generator Loss: {gen_loss:.4f}, Discriminator Loss: {disc_loss:.4f}")
        
        self.save_model()
    
    def save_samples(self, epoch, test_input):
        predictions = self.generator(test_input, training=False)
        
        fig = plt.figure(figsize=(12, 12))
        for i in range(min(4, len(predictions))):
            plt.subplot(2, 2, i+1)
            plt.imshow(predictions[i] * 0.5 + 0.5)
            plt.axis('off')
        
        save_path = os.path.join('samples', f'sample_epoch_{epoch:03d}.png')
        plt.savefig(save_path)
        plt.close()
    
    def save_model(self):
        self.generator.save(os.path.join('models', 'generator_model.keras'))
        self.discriminator.save(os.path.join('models', 'discriminator_model.keras'))


In [11]:
def create_video_from_images(image_folder, output_path, fps=30):
    if not os.path.exists(image_folder):
        raise ValueError(f"Image folder '{image_folder}' does not exist")
    
    images = [img for img in os.listdir(image_folder) if img.endswith(".png")]
    if not images:
        raise ValueError(f"No PNG images found in {image_folder}")
    
    images.sort()
    first_image_path = os.path.join(image_folder, images[0])
    frame = cv2.imread(first_image_path)
    if frame is None:
        raise ValueError(f"Could not read image: {first_image_path}")
    
    height, width, _ = frame.shape
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))
    
    print(f"Creating video from {len(images)} images...")
    for image in tqdm(images):
        frame = cv2.imread(os.path.join(image_folder, image))
        if frame is not None:
            out.write(frame)
    
    out.release()
    print(f"Video saved to {output_path}")


In [12]:
def main():
    try:
        create_directories()
        
        sharp_images, motion_blur_images, defocus_blur_images = load_and_preprocess_images(
            '/kaggle/input/blur-dataset/sharp',
            '/kaggle/input/blur-dataset/motion_blurred',
            '/kaggle/input/blur-dataset/defocused_blurred'
        )
        
        gan = BlurGAN()
        
        print("Training with motion blur...")
        gan.train(sharp_images, motion_blur_images, epochs=100)
        
        print("Training with defocus blur...")
        gan.train(sharp_images, defocus_blur_images, epochs=100)
        
    except Exception as e:
        print(f"An error occurred: {str(e)}")
        raise

if __name__ == "__main__":
    main()


Training with motion blur...
Epoch 1/100


100%|██████████| 11/11 [00:17<00:00,  1.63s/it]


Generator Loss: 5.1525, Discriminator Loss: 0.9147
Epoch 2/100


100%|██████████| 11/11 [00:09<00:00,  1.11it/s]


Generator Loss: 8.6403, Discriminator Loss: 1.6726
Epoch 3/100


...

Generator Loss: 3.7944, Discriminator Loss: 0.9304
Epoch 99/100


100%|██████████| 11/11 [00:09<00:00,  1.16it/s]


Generator Loss: 3.2701, Discriminator Loss: 0.5419
Epoch 100/100


100%|██████████| 11/11 [00:09<00:00,  1.16it/s]


Generator Loss: 6.0715, Discriminator Loss: 0.4153


In [14]:
create_video_from_images('samples', 'output_video.mp4', fps=90)
 

Creating video from 20 images...


100%|██████████| 20/20 [00:01<00:00, 17.28it/s]

Video saved to output_video.mp4
